##Import library & Load the dataset from gdrive

In [21]:
# Import all the necessary library & files!
import os
import numpy as np
import pandas as pd

from keras.models  import Sequential, Model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Initialization

In [0]:
image_path = './gdrive/My Drive/24hackton/images'
model_path = './gdrive/My Drive/24hackton/models'
train_img_path = '{}/train'.format(image_path)
val_img_path = '{}/val'.format(image_path)

## Data augmentation

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=25,
        vertical_flip=True,
        fill_mode='reflect',
        data_format='channels_last',
        brightness_range=[0.5, 1.5],
        featurewise_center=True,
        featurewise_std_normalization=True)

val_datagen = ImageDataGenerator(
    rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_img_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_img_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

In [0]:
print (train_generator.class_indices)
Labels = '\n'.join(sorted(train_generator.class_indices.keys()))
print (Labels)
print(train_generator[0][0].shape)

## Building of CNN Classification Model

In [0]:
def create_model(dropout= True):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                  activation='relu',
                  input_shape=(224, 224, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Flatten())
  model.add(Dense(1000, activation='relu'))
  model.add(Dense(train_generator.num_classes, activation='softmax'))

  return model

model = create_model()

# Save the best model with max val_acc using ModelCheckpoint
checkpoint1 = ModelCheckpoint("{}/model.h5".format(model_path), monitor='val_acc', verbose=3, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

# Define parameters
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()